In [1]:
from typing import Callable

from sinapses_cliente.modelo.requisicao.requisicao import Requisicao
from sinapses_cliente.modelo.resposta.resposta import Resposta

from sinapses_cliente.modelo.treinamento_resultado import TreinamentoResultado
from sinapses_cliente.modelo.servico_contexto import ServicoContexto
from sinapses_cliente.sinapses_pipeline import SinapsesPipeline
from sinapses_cliente.sinapses_sessao import SinapsesSessao

In [2]:
pipeline = SinapsesPipeline.instancia();

O arquivo de configuração foi encontrado em: /home/sinapses/curso/projetos/trata_texto/sinapses.yml
Validando as configurações do arquivo encontrado...
 - O endereço do sinapses é válido!
 - A autenticação é válida!
 - O id da versão do modelo é válida!
Arquivo de configuração válido!
Instanciando o cliente do Sinapses - Versão: 1.4.0


In [3]:
def funcao_treinamento(sessao: SinapsesSessao) -> TreinamentoResultado:
    return TreinamentoResultado(1, [])

In [4]:
import os
import sys
import pickle
import zipfile
from io import BytesIO
from pathlib import Path

def save_file(modelo, filename):
    f = open(filename, "wb")
    f.write(get_file(modelo, filename))
    f.close()
    
def get_file(modelo, filename):
    buffer = modelo.get_recurso_por_nome(filename).conteudo.bytes
    return buffer

def unzip(modelo, filename):
    z = zipfile.ZipFile(BytesIO(get_file(modelo, filename)))
    z.extractall('./')
    z.close()

In [5]:
def funcao_inicializacao_servico(sessao: SinapsesSessao):
    print('iniciando servico CGC UNIFOR')
    modelo = sessao.get_modelo_versao()
    
    save_file(modelo, 'main.py')
    save_file(modelo, 'Legis_Class.py')
    unzip(modelo, 'dicionario_legislacao.zip')

    print('servico GCC UNIFOR iniciado com sucesso')
    
    return ServicoContexto(
    )

In [6]:
def funcao_servico(sessao: SinapsesSessao, ctx: ServicoContexto, requisicao: Requisicao) -> Resposta:
    tipo = int(requisicao.tipo)
    lei = requisicao.lei
    
    ano = 0
    try:
        ano = int(requisicao.ano)
    except Exception as e:
        ano = 0

    artigo = requisicao.artigo
    arg_artigo = requisicao.complemento
    paragrafo = requisicao.paragrafo
    inciso = requisicao.inciso
    alinea = requisicao.alinea
    item = requisicao.item
    
    from main import consulta_texto_lei, dic_leis, busca_norma

    resposta = None
    try:
        legislacao = (tipo, lei, ano, artigo, arg_artigo, paragrafo, inciso, alinea, item)
        resultado = consulta_texto_lei(dic_leis, legislacao)        
        resposta = Resposta(resultado)
        return resposta
    except Exception as e: 
        return Resposta(str(e))

In [7]:
def funcao_teste_servico(_s: SinapsesSessao, _c: ServicoContexto, servico: Callable[[str], str]) -> None:    
    tipo = '0'
    lei = '12.587'
    ano = '2012'
    artigo = '17'
    complemento = '0'
    paragrafo = 'unico'
    inciso = '0'
    alinea = '0'
    item = '0'
    
    r1 = servico({'tipo': tipo, 'lei': lei, 'ano': ano, 'artigo': artigo, 'complemento': complemento, 
                  'paragrafo': paragrafo, 'inciso': inciso, 'alinea': alinea, 'item': item })
    print(r1)
    assert True == True

In [8]:
pipeline.definir_treinamento(funcao_treinamento)
pipeline.definir_inicializacao_servico(funcao_inicializacao_servico)
pipeline.definir_servico(funcao_servico)
pipeline.definir_teste_servico(funcao_teste_servico)

In [9]:
# pipeline.testar_pipeline()